In [2]:
import pandas as pd
import numpy as np

In [3]:
# gameplay_data = pd.read_excel("./Analytics Position Case Study.xlsx", "User Gameplay data", header = 3, dtype=str)
gameplay_data = pd.read_csv("./gameplay.csv")
gameplay_data.rename(columns = {'User ID':'User Id'}, inplace = True)
gameplay_data.head()

,User Id,Games Played,Datetime
0,851,1,01-10-2022 00:00
1,717,1,01-10-2022 00:00
2,456,1,01-10-2022 00:00
3,424,1,01-10-2022 00:00
4,845,1,01-10-2022 00:00


In [4]:
gameplay_data.isnull().sum()

User Id         0
Games Played    0
Datetime        0
dtype: int64

In [5]:
gameplay_data.dtypes

User Id          int64
Games Played     int64
Datetime        object
dtype: object

In [6]:
gameplay_data["Datetime"] = pd.to_datetime(gameplay_data["Datetime"], format="%d-%m-%Y %H:%M")

gameplay_data.tail()

,User Id,Games Played,Datetime
355261,658,1,2022-10-31 23:59:00
355262,582,1,2022-10-31 23:59:00
355263,272,1,2022-10-31 23:59:00
355264,563,1,2022-10-31 23:59:00
355265,289,1,2022-10-31 23:59:00


In [7]:
gameplay_data.dtypes

User Id                  int64
Games Played             int64
Datetime        datetime64[ns]
dtype: object

In [8]:
deposit_data = pd.read_csv("./deposit.csv")
deposit_data.rename(columns = {'Amount':'deposit'}, inplace = True)
deposit_data.head()

,User Id,Datetime,deposit
0,357,01-10-2022 00:03,2000
1,776,01-10-2022 00:03,2500
2,492,01-10-2022 00:06,5000
3,803,01-10-2022 00:07,5000
4,875,01-10-2022 00:09,1500


In [9]:
deposit_data.isnull().sum()

User Id     0
Datetime    0
deposit     0
dtype: int64

In [10]:
deposit_data.dtypes

User Id      int64
Datetime    object
deposit      int64
dtype: object

In [11]:
deposit_data["Datetime"] = pd.to_datetime(deposit_data["Datetime"], format = "%d-%m-%Y %H:%M")

deposit_data.head()

,User Id,Datetime,deposit
0,357,2022-10-01 00:03:00,2000
1,776,2022-10-01 00:03:00,2500
2,492,2022-10-01 00:06:00,5000
3,803,2022-10-01 00:07:00,5000
4,875,2022-10-01 00:09:00,1500


In [12]:
withrawl_data = pd.read_csv("./withrawl.csv")
withrawl_data.rename(columns = {'Amount':'withrawl'}, inplace = True)
withrawl_data.head()

,User Id,Datetime,withrawl
0,190,01-10-2022 00:03,5872
1,159,01-10-2022 00:16,9540
2,164,01-10-2022 00:24,815
3,946,01-10-2022 00:29,23000
4,763,01-10-2022 00:40,9473


In [13]:
withrawl_data.isnull().sum()

User Id     0
Datetime    0
withrawl    0
dtype: int64

In [14]:
withrawl_data.dtypes

User Id      int64
Datetime    object
withrawl     int64
dtype: object

In [15]:
withrawl_data["Datetime"] = pd.to_datetime(withrawl_data["Datetime"], format = "%d-%m-%Y %H:%M")

withrawl_data.head()

,User Id,Datetime,withrawl
0,190,2022-10-01 00:03:00,5872
1,159,2022-10-01 00:16:00,9540
2,164,2022-10-01 00:24:00,815
3,946,2022-10-01 00:29:00,23000
4,763,2022-10-01 00:40:00,9473


In [16]:
len(withrawl_data), len(deposit_data), len(gameplay_data)

(3566, 17438, 355266)

In [17]:
def slot(df_list):
    for i in df_list:
        i["slot"] = 0
        i.loc[(i["Datetime"].dt.time >=  pd.Timestamp('00:00:00').time()) & (i["Datetime"].dt.time <= pd.Timestamp('11:59:59').time()), "slot"] = 1
        i.loc[~((i["Datetime"].dt.time >=  pd.Timestamp('00:00:00').time()) & (i["Datetime"].dt.time <= pd.Timestamp('11:59:59').time())), "slot"] = 2

In [18]:
slot([gameplay_data, deposit_data, withrawl_data])

In [19]:
gameplay_data["Datetime"] = pd.to_datetime(gameplay_data["Datetime"]).dt.date
deposit_data["Datetime"] = pd.to_datetime(deposit_data["Datetime"]).dt.date
withrawl_data["Datetime"] = pd.to_datetime(withrawl_data["Datetime"]).dt.date

x = gameplay_data.groupby(["User Id", 'Datetime', 'slot'])['Games Played'].sum().reset_index()
# x["Datetime"] = pd.to_datetime(x["Datetime"]).dt.date

# y = deposit_data.groupby(["User Id", 'Datetime', "slot"])['deposit'].sum().reset_index()
y = deposit_data.groupby(["Datetime", "User Id", "slot"])['deposit'].sum().reset_index()
# y["Datetime"] = pd.to_datetime(y["Datetime"]).dt.date

z = withrawl_data.groupby(["User Id", 'Datetime', 'slot'])['withrawl'].sum().reset_index()
# z["Datetime"] = pd.to_datetime(z["Datetime"]).dt.date

In [20]:
x

,User Id,Datetime,slot,Games Played
0,0,2022-10-04,2,1
1,0,2022-10-05,2,1
2,0,2022-10-06,1,1
3,0,2022-10-10,1,1
4,0,2022-10-10,2,1
...,...,...,...,...
34009,999,2022-10-29,2,2
34010,999,2022-10-30,1,4
34011,999,2022-10-30,2,2
34012,999,2022-10-31,1,6


In [21]:
df0 = pd.merge(x.drop(["Games Played"], axis = 1), y, how = "outer")

# df0.to_csv("./test1.csv")

df1 = pd.merge(df0.drop(["deposit"], axis = 1), z, how = "outer")
df2 = pd.merge(df1.drop(["withrawl"], axis = 1), df0, how = "outer")

df3 = pd.concat([df1, df2["deposit"]], axis = 1)

df = pd.merge(df3.drop(["withrawl", "deposit"], axis = 1), x, how = "outer")
df = pd.concat([df, df3["withrawl"], df3["deposit"]], axis = 1)
df.head()

,User Id,Datetime,slot,Games Played,withrawl,deposit
0,0,2022-10-04,2,1.0,NaN,NaN
1,0,2022-10-05,2,1.0,NaN,NaN
2,0,2022-10-06,1,1.0,NaN,NaN
3,0,2022-10-10,1,1.0,NaN,NaN
4,0,2022-10-10,2,1.0,NaN,NaN


In [22]:
print(df.isnull().sum())

User Id             0
Datetime            0
slot                0
Games Played     5037
withrawl        35948
deposit         26691
dtype: int64


In [23]:
df.fillna(0, inplace = True)

In [24]:
#  Checking data for specific user for testing purpose 
df[df["User Id"] == 219]

,User Id,Datetime,slot,Games Played,withrawl,deposit
8054,219,2022-10-01,2,0.0,0.0,500.0
8055,219,2022-10-02,1,1.0,0.0,4000.0
8056,219,2022-10-04,1,1.0,0.0,0.0
8057,219,2022-10-04,2,2.0,0.0,0.0
8058,219,2022-10-07,1,1.0,0.0,0.0
8059,219,2022-10-07,2,2.0,0.0,0.0
8060,219,2022-10-09,1,0.0,0.0,2000.0
8061,219,2022-10-10,1,1.0,0.0,0.0
8062,219,2022-10-13,1,0.0,0.0,2000.0
8063,219,2022-10-14,2,1.0,0.0,0.0


# Part A

## Q1. Find Playerwise Loyalty points earned by Players in the following slots:-
    a. 2nd October Slot S1
    b. 16th October Slot S2
    c. 18th October Slot S1
    d. 26th October Slot S2

In [25]:
def loyalty_date_slot(date, slot):
    
    date = pd.Timestamp(date).date()
    
    deposits = df[(df["Datetime"] == date) & (df["slot"] == slot)]["deposit"]
    withrawl = df[(df["Datetime"] == date) & (df["slot"] == slot)]["withrawl"]
    games_played = df[(df["Datetime"] == date) & (df["slot"] == slot)]["Games Played"]
    
    deposit_l = 0.01 * deposits
    withrawl_l = 0.005 * withrawl
    freq_deposit_l  = 0.001 * np.maximum(deposits - withrawl, 0)
    game_played_l = 0.2 * games_played
    
    loyalty_point = deposit_l + withrawl_l + freq_deposit_l + game_played_l
    
    
    loyal_df = df[(df["Datetime"] == date) & (df["slot"] == slot)].copy()
    loyal_df["Loyalty point on {}".format(date)] = loyalty_point
    return loyal_df


### a. Loyalty points on 2nd October for Slot S1

In [26]:
loyalty_date_slot("2022-10-02", 1).drop(["Games Played", "withrawl", "deposit"], axis = 1)

,User Id,Datetime,slot,Loyalty point on 2022-10-02
23,2,2022-10-02,1,0.4
131,5,2022-10-02,1,8.4
193,6,2022-10-02,1,44.4
234,7,2022-10-02,1,1.0
273,8,2022-10-02,1,1.2
...,...,...,...,...
38693,990,2022-10-02,1,14.0
38785,992,2022-10-02,1,8.4
38875,996,2022-10-02,1,2.2
38937,997,2022-10-02,1,1.0


### b. Loyalty points on 16th October for Slot 2

In [27]:
loyalty_date_slot("2022-10-16", 2).drop(["Games Played", "withrawl", "deposit"], axis = 1)

,User Id,Datetime,slot,Loyalty point on 2022-10-16
51,2,2022-10-16,2,0.4
160,5,2022-10-16,2,12.8
213,6,2022-10-16,2,0.2
302,8,2022-10-16,2,1.0
364,9,2022-10-16,2,32.2
...,...,...,...,...
38722,990,2022-10-16,2,12.4
38768,991,2022-10-16,2,66.2
38814,992,2022-10-16,2,8.8
38904,996,2022-10-16,2,1.8


### c. Loyalty point on 18th October for Slot S1

In [28]:
loyalty_date_slot("2022-10-18", 1).drop(["Games Played", "withrawl", "deposit"], axis = 1)

,User Id,Datetime,slot,Loyalty point on 2022-10-18
54,2,2022-10-18,1,0.4
101,3,2022-10-18,1,0.4
163,5,2022-10-18,1,1.6
253,7,2022-10-18,1,0.6
305,8,2022-10-18,1,1.8
...,...,...,...,...
38725,990,2022-10-18,1,14.4
38817,992,2022-10-18,1,9.4
38907,996,2022-10-18,1,9.8
38963,997,2022-10-18,1,0.2


### d. Loyalty point on 26th October for Slot S2

In [29]:
loyalty_date_slot("2022-10-26", 2).drop(["Games Played", "withrawl", "deposit"], axis = 1)

,User Id,Datetime,slot,Loyalty point on 2022-10-26
71,2,2022-10-26,2,990.000
180,5,2022-10-26,2,11.800
225,6,2022-10-26,2,0.200
263,7,2022-10-26,2,33.285
322,8,2022-10-26,2,1.400
...,...,...,...,...
38834,992,2022-10-26,2,158.800
38867,995,2022-10-26,2,0.200
38924,996,2022-10-26,2,0.800
38978,997,2022-10-26,2,0.400


## Q2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October.  In case of tie, number of games played should be taken as the next criteria for ranking .

In [30]:
#  Making new dat frame for loyalti points for all users

def loyalty():
    
    # date = pd.Timestamp(date).date()
    
    deposits = df["deposit"]
    withrawl = df["withrawl"]
    games_played = df["Games Played"]
    
    deposit_l = 0.01 * deposits
    withrawl_l = 0.005 * withrawl
    freq_deposit_l  = 0.001 * np.maximum(deposits - withrawl, 0)
    game_played_l = 0.2 * games_played
    
    loyalty_point = deposit_l + withrawl_l + freq_deposit_l + game_played_l
    
    
    loyal_df = df.copy()
    loyal_df["Loyalty Point"] = loyalty_point
    return loyal_df

df_2 = loyalty()

In [31]:
df_2.head()

,User Id,Datetime,slot,Games Played,withrawl,deposit,Loyalty Point
0,0,2022-10-04,2,1.0,0.0,0.0,0.2
1,0,2022-10-05,2,1.0,0.0,0.0,0.2
2,0,2022-10-06,1,1.0,0.0,0.0,0.2
3,0,2022-10-10,1,1.0,0.0,0.0,0.2
4,0,2022-10-10,2,1.0,0.0,0.0,0.2


## Overall Loyalty Points for all users

In [32]:
all_user_loyalty = df_2.groupby(["User Id"])["Loyalty Point"].sum().reset_index()
all_user_game_played = df_2.groupby(["User Id"])["Games Played"].sum().reset_index()

all_user_loyalty.head()

,User Id,Loyalty Point
0,0,3.000
1,1,56.600
2,2,12552.475
3,3,456.000
4,4,20.250


In [33]:
all_user_game_played.head()

,User Id,Games Played
0,0,15.0
1,1,8.0
2,2,97.0
3,3,80.0
4,4,5.0


In [34]:
rankings = all_user_loyalty.sort_values(by = "Loyalty Point", ascending=False)
rankings.head()


,User Id,Loyalty Point
634,634,83943.325
672,672,24833.950
99,99,24515.849
212,212,24124.241
566,566,20937.880


In [35]:
# getting rows where loyalty points are repeated for further opertions

same_point = rankings.loc[rankings["Loyalty Point"].duplicated()]
same_point.head()


,User Id,Loyalty Point
724,724,353.8
877,877,275.6
79,79,167.6
711,711,167.6
261,261,146.6


In [36]:
games = []
for i in same_point["User Id"]:
    x = all_user_game_played[all_user_game_played["User Id"] == i]["Games Played"]
    games.append(x.values[0])
print(games)

[174.0, 278.0, 13.0, 13.0, 53.0, 14.0, 497.0, 8.0, 7.0, 35.0, 34.0, 25.0, 3.0, 223.0, 8.0, 30.0, 14.0, 9.0, 9.0, 8.0, 3.0, 1.0, 1.0, 1.0, 1.0, 7.0, 19.0, 69.0, 68.0, 3.0, 26.0, 15.0, 12.0, 8.0, 6.0, 5.0, 5.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0]


In [37]:
handle_same_values = same_point.copy()
games_df =  pd.DataFrame(games, columns = ["games played"])
games_df.head()

,games played
0,174.0
1,278.0
2,13.0
3,13.0
4,53.0


In [38]:
rankings2 = pd.concat([same_point.reset_index(drop=True), games_df.reset_index(drop=True)], axis = 1)
rankings2.head()

,User Id,Loyalty Point,games played
0,724,353.8,174.0
1,877,275.6,278.0
2,79,167.6,13.0
3,711,167.6,13.0
4,261,146.6,53.0


# Rankings 

In [39]:
final_dummy = pd.merge(rankings,  rankings2, how = "outer")
final_dummy.fillna(0, inplace = True)

final_dummy["Loyalty Point"] = final_dummy["Loyalty Point"] + final_dummy["games played"]
final_dummy = final_dummy.drop(["games played"], axis = 1)

final_dummy = final_dummy.sort_values(by = "Loyalty Point", ascending=False).reset_index(drop = True)
final_dummy["Rank"] = range(1, len(final_dummy) + 1)
final_dummy


,User Id,Loyalty Point,Rank
0,634,83943.325,1
1,672,24833.950,2
2,99,24515.849,3
3,212,24124.241,4
4,566,20937.880,5
...,...,...,...
995,589,1.200,996
996,243,1.000,997
997,165,0.600,998
998,993,0.400,999


# Q3. What is the average deposit amount?

In [40]:
deposit_data_per_user = df.groupby(["User Id"])["deposit"].sum().reset_index()
# deposit_data_per_user["deposit"]
average_deposits = deposit_data_per_user["deposit"].mean()
print("Average Deposit is: " + str(average_deposits))

Average Deposit is: 95772.729


# Q4. What is the average deposit amount per user in a month?

In [41]:
dq14 = df.copy()

avg_depo_user = dq14.groupby(["User Id"])["deposit"].mean().reset_index()
avg_depo_user.rename(columns = {'deposit':'Average Deposit in a month'}, inplace = True)
avg_depo_user

,User Id,Average Deposit in a month
0,0,0.000000
1,1,625.000000
2,2,9775.862069
3,3,909.090909
4,4,291.666667
...,...,...
995,995,1815.789474
996,996,1554.838710
997,997,0.000000
998,998,925.000000


# Q5. What is the average number of games played per user?

In [42]:
dfq5 = df.copy()

avg_games_user = dfq5.groupby(["User Id"])["Games Played"].mean().reset_index()
avg_games_user.rename(columns = {'Games Played':'Average Games Played in a month'}, inplace = True)
avg_games_user

,User Id,Average Games Played in a month
0,0,1.153846
1,1,1.000000
2,2,1.672414
3,3,1.818182
4,4,0.833333
...,...,...
995,995,0.947368
996,996,7.645161
997,997,2.180000
998,998,0.500000


# Part B

# Q1.  find out which 50 players are at the top of the leaderboard

In [43]:
top_50 = final_dummy.copy()
top_50 = top_50.drop(["Loyalty Point"], axis = 1)[:50]
top_50

,User Id,Rank
0,634,1
1,672,2
2,99,3
3,212,4
4,566,5
5,740,6
6,714,7
7,421,8
8,30,9
9,369,10


# Q2. How much bonus money should be given to the players. Should they base it on the amount of loyalty points? Should it be based on number of games? Or something else?


### Answer.  In my opinion the price distrubution will be based on Loyalty Points and activeness of the player (frequency of deposits, games played, and withrawls of each day/week)

# Q3. Suggest a suitable way to divide the allocated money keeping in mind the following point that oOnly top 50 ranked players are awarded bonus

### Answer. We can divide the price amount as per importance of ranking, like we can allot 5% of amount to top 3 players then 2.5% amount to 4th to 10th Player then distribute the remaining amount equally to remaing players (11 - 50)

In [44]:
prize_amount =  50000
top3_amount = (prize_amount/100) * 5
# prize_amount =  50000 - (3*(top3_amount))
top10_amount = (prize_amount/100) * 2.5
prize_amount = prize_amount - ((3*(top3_amount)) + (7*(top10_amount)))
remaing_user_amount = prize_amount/40

prize_dist = []
for i in range(50):
    if i<=2:
        prize_dist.append(top3_amount)
    elif i<= 9:
        prize_dist.append(top10_amount)
    else:
        prize_dist.append(remaing_user_amount)

prize_dist

top_50["Prize Amount"] = pd.DataFrame(prize_dist)
top_50

,User Id,Rank,Prize Amount
0,634,1,2500.00
1,672,2,2500.00
2,99,3,2500.00
3,212,4,1250.00
4,566,5,1250.00
5,740,6,1250.00
6,714,7,1250.00
7,421,8,1250.00
8,30,9,1250.00
9,369,10,1250.00


# Part C

# Q1. Would you say the loyalty point formula is fair or unfair?

### Yes, loyalty point is completly fair

# Q2.  Can you suggest any way to make the loyalty point formula more robust

### There could be few more customer's actions to determine loyalty point like:
### 1. Loyalty points based on user activity
### 2. Loyalty points based on succesful application refrence to new users given by existing user
### 3. Loyalty points based on popularity of player (if there israting system)
### 4. Loyalty points for old players

